# Piecewise Linear Constraints

This notebook demonstrates linopy's three PWL formulations. Each example
builds a separate dispatch model where a single power plant must meet
a time-varying demand.

| Example | Plant | Limitation | Formulation |
|---------|-------|------------|-------------|
| 1 | Gas turbine (0–100 MW) | Convex heat rate | SOS2 |
| 2 | Coal plant (0–150 MW) | Monotonic heat rate | Incremental |
| 3 | Diesel generator (off or 50–80 MW) | Forbidden zone | Disjunctive |

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

import linopy

time = pd.Index([1, 2, 3], name="time")


def plot_pwl_results(model, breakpoints, demand, color="C0", fuel_rate=None):
    """Plot PWL curve with operating points and dispatch vs demand."""
    sol = model.solution
    bp = breakpoints.to_pandas()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3.5))

    # Left: PWL curve with operating points
    if "var" in breakpoints.dims:
        # Connected: power-fuel curve from var dimension
        ax1.plot(
            bp.loc["power"], bp.loc["fuel"], "o-", color=color, label="Breakpoints"
        )
        for t in time:
            ax1.plot(
                sol["power"].sel(time=t),
                sol["fuel"].sel(time=t),
                "s",
                ms=10,
                label=f"t={t}",
            )
        ax1.set(xlabel="Power (MW)", ylabel="Fuel (MWh)", title="Heat rate curve")
    else:
        # Disconnected: segments with linear cost
        for seg in bp.index:
            lo, hi = bp.loc[seg]
            pw = [lo, hi] if lo != hi else [lo]
            ax1.plot(
                pw,
                [fuel_rate * p for p in pw],
                "o-",
                color=color,
                label="Breakpoints" if seg == 0 else None,
            )
        ax1.axvspan(
            bp.iloc[0, 1] + 0.5,
            bp.iloc[1, 0] - 0.5,
            color="red",
            alpha=0.1,
            label="Forbidden zone",
        )
        for t in time:
            p = float(sol["power"].sel(time=t))
            ax1.plot(p, fuel_rate * p, "s", ms=10, label=f"t={t}")
        ax1.set(xlabel="Power (MW)", ylabel="Cost", title="Cost curve")
    ax1.legend()

    # Right: dispatch vs demand
    x = list(range(len(time)))
    power_vals = sol["power"].values
    ax2.bar(x, power_vals, color=color, label="Power")
    if "backup" in sol:
        ax2.bar(
            x,
            sol["backup"].values,
            bottom=power_vals,
            color="C3",
            alpha=0.5,
            label="Backup",
        )
    ax2.step(
        [v - 0.5 for v in x] + [x[-1] + 0.5],
        list(demand.values) + [demand.values[-1]],
        where="post",
        color="black",
        lw=2,
        label="Demand",
    )
    ax2.set(
        xlabel="Time", ylabel="MW", title="Dispatch", xticks=x, xticklabels=time.values
    )
    ax2.legend()
    plt.tight_layout()

## 1. SOS2 formulation — Gas turbine

The gas turbine has a **convex** heat rate: efficient at moderate load,
increasingly fuel-hungry at high output. We use the **SOS2** formulation
to link power output and fuel consumption.

In [ ]:
breakpoints = linopy.breakpoints(power=[0, 30, 60, 100], fuel=[0, 36, 84, 170])
breakpoints.to_pandas()

In [ ]:
m1 = linopy.Model()

power = m1.add_variables(name="power", lower=0, upper=100, coords=[time])
fuel = m1.add_variables(name="fuel", lower=0, coords=[time])

# breakpoints are auto-broadcast to match the time dimension
m1.add_piecewise_constraints(
    {"power": power, "fuel": fuel},
    breakpoints,
    dim="breakpoint",
    name="pwl",
    method="sos2",
)

demand1 = xr.DataArray([50, 80, 30], coords=[time])
m1.add_constraints(power >= demand1, name="demand")
m1.add_objective(fuel.sum())

In [ ]:
m1.solve()

In [ ]:
m1.solution[["power", "fuel"]].to_pandas()

In [ ]:
plot_pwl_results(m1, breakpoints, demand1, color="C0")

## 2. Incremental formulation — Coal plant

The coal plant has a **monotonically increasing** heat rate. Since all
breakpoints are strictly monotonic, we can use the **incremental**
formulation — a pure LP with no SOS2 or binary variables.

In [ ]:
breakpoints = linopy.breakpoints(power=[0, 50, 100, 150], fuel=[0, 55, 130, 225])
breakpoints.to_pandas()

In [ ]:
m2 = linopy.Model()

power = m2.add_variables(name="power", lower=0, upper=150, coords=[time])
fuel = m2.add_variables(name="fuel", lower=0, coords=[time])

# breakpoints are auto-broadcast to match the time dimension
m2.add_piecewise_constraints(
    {"power": power, "fuel": fuel},
    breakpoints,
    dim="breakpoint",
    name="pwl",
    method="incremental",
)

demand2 = xr.DataArray([80, 120, 50], coords=[time])
m2.add_constraints(power >= demand2, name="demand")
m2.add_objective(fuel.sum())

In [ ]:
m2.solve();

In [ ]:
m2.solution[["power", "fuel"]].to_pandas()

In [ ]:
plot_pwl_results(m2, breakpoints, demand2, color="C1")

## 3. Disjunctive formulation — Diesel generator

The diesel generator has a **forbidden operating zone**: it must either
be off (0 MW) or run between 50–80 MW. Because of this gap, we add a
high-cost **backup** source to cover demand when the diesel is off or at
its maximum.

In [ ]:
breakpoints = linopy.breakpoints.segments([(0, 0), (50, 80)])
breakpoints.to_pandas()

In [ ]:
m3 = linopy.Model()

power = m3.add_variables(name="power", lower=0, upper=80, coords=[time])
backup = m3.add_variables(name="backup", lower=0, coords=[time])

# breakpoints are auto-broadcast to match the time dimension
m3.add_disjunctive_piecewise_constraints(power, breakpoints, name="pwl")

demand3 = xr.DataArray([10, 70, 90], coords=[time])
m3.add_constraints(power + backup >= demand3, name="demand")
m3.add_objective((2.5 * power + 10 * backup).sum())

In [ ]:
m3.solve()

In [ ]:
m3.solution[["power", "backup"]].to_pandas()

In [ ]:
plot_pwl_results(m3, breakpoints, demand3, color="C2", fuel_rate=2.5)